# Using Knowledge tools

**Step 1**: Install NuGet packages

To get started with Azure AI Foundry, you need to install the required NuGet packages:

In [34]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7"
#r "nuget: Microsoft.SemanticKernel.Agents.Core"
#r "nuget: DotNetEnv, 3.1.0"
#r "nuget: Microsoft.SemanticKernel.Agents.AzureAI, 1.56.0-preview"

Installed Packages DotNetEnv, 3.1.0 Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7 Microsoft.SemanticKernel, 1.56.0 Microsoft.SemanticKernel.Agents.AzureAI, 1.56.0-preview Microsoft.SemanticKernel.Agents.Core, 1.56.0

**Step 2**: Read environment variables

In this step, we load these variables from a `.env` file (if present) so that they can be accessed by the application.

In [35]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Environment.CurrentDirectory, "../..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from {envFilePath}");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

Loaded environment variables from d:\personal\aiagent-workshop\notebooks\ai-foundry\../..\.env


**Step 3**: Instantiate the Project client

Create an instance of the `PersistentAgentsClient` using the endpoint URL and the Azure AI Foundry project.

Note that **DefaultAzureCredential** is used for authentication. To successfully authenticate, log in to the Azure CLI using `az login --use-device-code`.

In [36]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;
using Azure.Identity;

var projectEndpoint = Environment.GetEnvironmentVariable("AZURE_AI_PROJECT_ENDPOINT");
#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
var client = AzureAIAgent.CreateAgentsClient(projectEndpoint, new DefaultAzureCredential());
#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

# Contoso Bikes Policy Q&A

This section demonstrates how to use the knowledge tools to answer questions about Contoso Bikes' company policies, such as the return policy.

**Step 1**: Define your agent

In this step, we define the attributes for the agent, including its name, instructions, tools (if any), and the model to be used. 

In [ ]:
var modelId = Environment.GetEnvironmentVariable("MODEL");
var policyAgentName = "ContosoPolicyAssistant";
var policyAgentInstructions = @"
    You are a helpful assistant for Contoso Bikes.
    Answer questions about company policies, especially the return policy, using the provided documentation.
    If a question is not covered by the documentation, politely inform the user.
";

// Upload the Contoso Bikes policy markdown file and create a vector store
var policyFilePath = Path.Combine(Environment.CurrentDirectory, "../../resources/contoso-bikes.pdf");
var policyFileStream = File.OpenRead(policyFilePath);

PersistentAgentFileInfo policyFileInfo = await client.Files.UploadFileAsync(policyFileStream, PersistentAgentFilePurpose.Agents, "contoso-bikes.pdf");
PersistentAgentsVectorStore policyFileStore = await client.VectorStores.CreateVectorStoreAsync([policyFileInfo.Id]);

var agentDefinition = await client.Administration.CreateAgentAsync(
    modelId,
    name: policyAgentName,
    instructions: policyAgentInstructions,
    tools: [new FileSearchToolDefinition()],
    toolResources: new()
    {
        FileSearch = new()
        {
            VectorStoreIds = { policyFileStore.Id },
        }
    });

**Step 2**: Create the agent

Finally, we create the agent using the `CreateAgentAsync` method of the `PersistentAgentsClient`. This method takes the agent definition and returns the created agent.

In [38]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;

#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
       
var agent = new AzureAIAgent(agentDefinition, client);

#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

**Step 3**: Creat a new thread for the agent

In [39]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;

#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
       
var thread = new AzureAIAgentThread(client);

#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

**Step 4** : Create a new message for the agent

Use the agent to answer questions about the return policy or other company policies.

In [40]:
#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
var policyThread = new AzureAIAgentThread(client);

var invokeAgentAsync = async (AzureAIAgentThread agentThread, string input) =>
{
    ChatMessageContent message = new(AuthorRole.User, input);
    await foreach (ChatMessageContent response in agent.InvokeAsync(message, agentThread))
    {
        Console.WriteLine($"{response.Role}: {response.Content}");
    }
};

await invokeAgentAsync(policyThread, "What is the return policy at Contoso Bikes?");
#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

assistant: At Contoso Bikes, the return policy is as follows:

- **Timeframe**: Most items can be returned within 30 days of delivery for a full refund or exchange.
- **Eligibility**: Items must be unused, in their original packaging, and accompanied by proof of purchase (e.g., receipt).
- **Exclusions**: Custom bikes, gift cards, and clearance items are not eligible for return.
- **Damaged or Defective Goods**: If the received item is damaged or defective, contact support within 7 days for a replacement or refund at no extra cost.
- **Return Process**:
  1. Contact the customer service team via email at support@contosobikes.com or by calling 1-800-BIKE-123 to initiate the return.
  2. An RMA (Return Merchandise Authorization) number and shipping instructions will be provided.
  3. Package the item securely, include all original materials, and mark the RMA number on the package.
  4. Use a trackable shipping service to ship the item to the provided address.
- **Refunds**: Processed wit

Ask another question to the agent to see how it responds based on the provided context.

In [41]:
await invokeAgentAsync(policyThread, "Are there any items that cannot be returned?");

assistant: Yes, the following items cannot be returned at Contoso Bikes:

1. **Custom Bikes**: Any bikes customized to a customer's specifications.
2. **Gift Cards**: These are non-returnable.
3. **Clearance Items**: Items purchased on clearance are not eligible for returns.

These exclusions are part of the company’s return policy【4:0†source】.


**Step 5**: Clean up

Delete the agent and the thread to clean up resources.

In [43]:
await policyThread.DeleteAsync();
await client.Administration.DeleteAgentAsync(agent.Id);